In [1]:
#pip install -q datasets

In [2]:
#pip install -q evaluate

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import Dataset
import evaluate
import numpy as np
from sklearn.model_selection import KFold
import torch

In [2]:
import transformers
print(transformers.__version__)

4.56.1


In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

In [4]:
df = pd.read_csv("clickbait_title_classification.csv")
df = df.rename(columns={"clickbait": "labels"})

In [5]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [6]:
def tokenize_fn(examples):
    return tokenizer(
        examples["title"],
        padding="max_length",
        truncation=True,
        max_length=128,
    )

In [7]:
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")
roc_auc = evaluate.load("roc_auc")

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    # Convert logits to predicted class
    preds = np.argmax(logits, axis=-1)

    # Convert logits to probabilities for ROC AUC
    probs = 1 / (1 + np.exp(-logits[:, 1]))  # sigmoid on class 1 logits

    return {
        "accuracy": accuracy.compute(predictions=preds, references=labels)["accuracy"],
        "precision": precision.compute(predictions=preds, references=labels, average="binary")["precision"],
        "recall": recall.compute(predictions=preds, references=labels, average="binary")["recall"],
        "f1": f1.compute(predictions=preds, references=labels, average="binary")["f1"],
        "roc_auc": roc_auc.compute(prediction_scores=probs, references=labels)["roc_auc"]
    }

In [10]:
#pip show transformers accelerate torch

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

for param in model.bert.parameters():
        param.requires_grad = False
    
for name, param in model.named_parameters():
    print(name, param.requires_grad)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


bert.embeddings.word_embeddings.weight False
bert.embeddings.position_embeddings.weight False
bert.embeddings.token_type_embeddings.weight False
bert.embeddings.LayerNorm.weight False
bert.embeddings.LayerNorm.bias False
bert.encoder.layer.0.attention.self.query.weight False
bert.encoder.layer.0.attention.self.query.bias False
bert.encoder.layer.0.attention.self.key.weight False
bert.encoder.layer.0.attention.self.key.bias False
bert.encoder.layer.0.attention.self.value.weight False
bert.encoder.layer.0.attention.self.value.bias False
bert.encoder.layer.0.attention.output.dense.weight False
bert.encoder.layer.0.attention.output.dense.bias False
bert.encoder.layer.0.attention.output.LayerNorm.weight False
bert.encoder.layer.0.attention.output.LayerNorm.bias False
bert.encoder.layer.0.intermediate.dense.weight False
bert.encoder.layer.0.intermediate.dense.bias False
bert.encoder.layer.0.output.dense.weight False
bert.encoder.layer.0.output.dense.bias False
bert.encoder.layer.0.output.Lay

In [9]:
kf = KFold(n_splits=10, shuffle=True, random_state=42)
fold_results = []

for fold, (train_val_index, test_index) in enumerate(kf.split(df)):
    print(f"=== Fold {fold+1} ===")

    test_df = df.iloc[test_index]

    # Remaining 9 folds for training+validation
    train_val_df = df.iloc[train_val_index]

    # --------------------------
    # Internal validation fold (8/1 split)
    # --------------------------
    train_df, val_df = train_test_split(train_val_df, test_size=1/9, random_state=42, stratify=train_val_df["labels"])

    # --------------------------
    # Convert to Hugging Face Datasets
    # --------------------------
    train_ds = Dataset.from_pandas(train_df)
    val_ds   = Dataset.from_pandas(val_df)
    test_ds  = Dataset.from_pandas(test_df)

    # --------------------------
    # Tokenize datasets
    # --------------------------
    train_ds = train_ds.map(tokenize_fn, batched=True)
    val_ds   = val_ds.map(tokenize_fn, batched=True)
    test_ds  = test_ds.map(tokenize_fn, batched=True)

    for ds in [train_ds, val_ds, test_ds]:
        ds.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

    #----------------------------------------------------------------------------------    
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    for param in model.bert.parameters():
        param.requires_grad = False
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    #-----------------------------------------------------------------------------------

    training_args = TrainingArguments(
        output_dir=f"./results/fold_{fold+1}",
        num_train_epochs=3,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        eval_strategy="epoch",
        save_strategy="epoch",
        learning_rate=2e-4,
        weight_decay=0.01,
        load_best_model_at_end=True,
        logging_steps=10,
        fp16=True,
        report_to="none",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    trainer.train()
    metrics = trainer.evaluate(test_ds)
    fold_results.append(metrics)
    print(f"\n Fold {fold+1} complete.")

=== Fold 1 ===


Map:   0%|          | 0/25600 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_1365397/534098796.py:57: FutureWarning: `tok

...

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Map:   0%|          | 0/3200 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_1365397/534098796.py:57: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,0.234700,0.239092,0.911563,0.942953,0.876482,0.908503,0.975241
2,0.191400,0.186116,0.934063,0.932298,0.936369,0.934329,0.982157
3,0.233200,0.179043,0.936250,0.938009,0.934498,0.936250,0.983390



 Fold 10 complete.


In [10]:
results_df = pd.DataFrame(fold_results)
print(results_df.mean())

eval_loss                     0.167488
eval_accuracy                 0.942469
eval_precision                0.944246
eval_recall                   0.940520
eval_f1                       0.942363
eval_roc_auc                  0.986352
eval_runtime                  3.286270
eval_samples_per_second    1047.401700
eval_steps_per_second        65.462700
epoch                         3.000000
dtype: float64


In [11]:
metrics_names = ["eval_accuracy", "eval_precision", "eval_recall", "eval_f1", "eval_roc_auc"]

for metric in metrics_names:
    values = [fold[metric] for fold in fold_results]
    avg = np.mean(values)
    std = np.std(values)
    print(f"{metric}: {avg:.4f} ± {std:.4f}")


eval_accuracy: 0.9425 ± 0.0045
eval_precision: 0.9442 ± 0.0049
eval_recall: 0.9405 ± 0.0066
eval_f1: 0.9424 ± 0.0043
eval_roc_auc: 0.9864 ± 0.0020
